In [ ]:
TOOLS="/groups/kemi/jmni/software/orca-external-tools"

import os, socket, subprocess, time, textwrap
from contextlib import contextmanager
from pathlib import Path
import pandas as pd
from pandas import Series
import signal

def _free_local_port() -> int:
    s = socket.socket(); s.bind(("127.0.0.1", 0))
    port = s.getsockname()[1]; s.close(); return port

@contextmanager
def uma_server(task: str, log_dir: str = ".", use_gpu: bool = False):
    port = _free_local_port()
    env = os.environ.copy()
    n = str(env.get("SLURM_CPUS_PER_TASK", "12"))
    env["OMP_NUM_THREADS"] = n
    env["VECLIB_MAXIMUM_THREADS"] = n
    env.setdefault("HF_HOME", os.path.expanduser("~/.cache/huggingface"))
    env.pop("MPLBACKEND", None)
    env["MPLBACKEND"] = "Agg"
    env["CUDA_VISIBLE_DEVICES"] = "" if not use_gpu else env.get("CUDA_VISIBLE_DEVICES", "0")
    Path(log_dir).mkdir(parents=True, exist_ok=True)
    log_path = Path(log_dir) / f"umaserver_{port}.log"
    logf = open(log_path, "wb")
    cmd = [f"{TOOLS}/umaserver.sh", "-m", task, "-b", f"127.0.0.1:{port}", "-n", "1"]
    p = subprocess.Popen(cmd, stdout=logf, stderr=subprocess.STDOUT, env=env, close_fds=True)
    ready = False
    for _ in range(120):
        try:
            with socket.create_connection(("127.0.0.1", port), timeout=0.5):
                ready = True; break
        except OSError:
            time.sleep(1)
    if not ready:
        p.terminate(); logf.close()
        raise RuntimeError(f"UMA server failed to start. See log: {log_path}")
    try:
        yield port, str(log_path)
    finally:
        try:
            p.terminate(); p.wait(timeout=10)
        except Exception:
            p.kill()
        logf.flush(); logf.close()

In [ ]:
from tooltoad.orca import orca_calculate

def UMA_job(atoms, coords, n_cores, memory):

    task = "omol@uma-m-1p1"

    with uma_server(task=task, log_dir="UMA-logs") as (port, _slog):
        client_block = f"""
        %method
        ProgExt "{TOOLS}/umaclient.sh"
        Ext_Params "-b 127.0.0.1:{port}"
        end
        %output
        Print[P_EXT_OUT] 1
        Print[P_EXT_GRAD] 1
        end
        """.strip()

    results = orca_calculate(
        atoms,
        coords,
        options={"ExtOpt": None, "NumFreq": None},
        xtra_inp_str=client_block,
        save_dir="UMA-test-1",
        save_files=["orca.out"],
        n_cores=10,
        memory=30,
    )

In [ ]:
import submitit
from tooltoad.chemutils import xyz2ac

f = Path("structures/ligandBpin_guess.xyz")

with open(f, 'r') as f:
    file = f.read()
    atoms, coords = xyz2ac(file)

JOB_NAME        = "my_UMA_job"
N_CORES         = 8
MEM_GB          = 30
TIMEOUT_MIN     = 14400
RESULTS_DIR     = "results"

executor = submitit.AutoExecutor(f"logs/{RESULTS_DIR}")
executor.update_parameters(
    slurm_partition="kemi1",
    cpus_per_task=N_CORES,
    mem_gb=MEM_GB,
    timeout_min=TIMEOUT_MIN,
)

executor.update_parameters(slurm_job_name=JOB_NAME)
executor.submit(UMA_job, atoms, coords, N_CORES, MEM_GB)
print(f"Submitted: {JOB_NAME}")